In this notebook, I aim to load the data saved from previous scraping notebooks and explore a way to pivot it in order to construct a final, unified DataFrame. This DataFrame will contain transplant data across multiple years for all transplant centers in Italy.

In [1]:
import pandas as pd 

In [2]:
# import one file
df1= pd.read_csv('../data_raw/2024/2024_Cuore.csv')
df1.head()

,Struttura trapianto,Cuore,Cuore - fegato,Totale Trapianti
0,"TO - AOU Città della Salute, PO OIRM",5,0,5
1,"TO - AOU Città della Salute, PO S.G.Battista",26,2,28
2,BG - OSPEDALE PAPA GIOVANNI XXIII - BERGAMO,22,0,22
3,MI - AO NIGUARDA CA' GRANDA - MILANO,35,0,35
4,PV - OSPEDALE POLICLINICO S. MATTEO - PAVIA,15,0,15


In [3]:
# removing the 'Totale Trapianti' column
df1.drop(labels='Totale Trapianti', axis=1, inplace=True)
df1.tail()

,Struttura trapianto,Cuore,Cuore - fegato
12,NA - A. O. MONALDI,20,0
13,BA - AOU CONSORZIALE POLICLINICO di BARI,73,0
14,PA - Is.Me.T.T.,40,0
15,CA - AZIENDA OSPEDALIERA G. BROTZU,6,0
16,Totale,411,2


In [4]:
# removing the last row with 'Totale' in the 'Struttura trapianto' column
df1.drop(labels=df1.index[-1], inplace=True)
df1.tail()

,Struttura trapianto,Cuore,Cuore - fegato
11,RM - OSPEDALE PEDIATRICO BAMBINO GESU',8,0
12,NA - A. O. MONALDI,20,0
13,BA - AOU CONSORZIALE POLICLINICO di BARI,73,0
14,PA - Is.Me.T.T.,40,0
15,CA - AZIENDA OSPEDALIERA G. BROTZU,6,0


In [5]:
# pivot the DataFrame from wide to long format (the desired format)
df1 = pd.melt(df1, id_vars= ['Struttura trapianto'], var_name='Organo', value_name='Numero')
    
# Adding Year to the dataframe
df1['Anno'] = 2024
df1.head()

,Struttura trapianto,Organo,Numero,Anno
0,"TO - AOU Città della Salute, PO OIRM",Cuore,5,2024
1,"TO - AOU Città della Salute, PO S.G.Battista",Cuore,26,2024
2,BG - OSPEDALE PAPA GIOVANNI XXIII - BERGAMO,Cuore,22,2024
3,MI - AO NIGUARDA CA' GRANDA - MILANO,Cuore,35,2024
4,PV - OSPEDALE POLICLINICO S. MATTEO - PAVIA,Cuore,15,2024


In [6]:
# Load the CSV file for 2024 - Cuore
df = pd.read_csv('../data_raw/2024/2024_Rene.csv')

# Drop the 'Totale Trapianti' column if it exists
if 'Totale Trapianti' in df.columns:
    df.drop(columns='Totale Trapianti', inplace=True)

# Drop the last row if it contains the total summary
if df.iloc[-1, 0].strip().lower() in ['totale', 'totali']:
    df.drop(index=df.index[-1], inplace=True)

df['Città'] = df['Struttura trapianto'].str.split('-').str[0].str.strip()

# Reshape the DataFrame from wide to long format
df_long = pd.melt(
    df,
    id_vars=['Struttura trapianto', 'Città'],
    var_name='Sottotipo',
    value_name='Numero'
)

# Add metadata columns
df_long['Organo'] = 'Rene'
df_long['Anno'] = 2024

# Optional: Reorder columns
df_long = df_long[['Struttura trapianto','Città', 'Organo', 'Sottotipo', 'Numero', 'Anno']]

# Preview
df_long.head()


,Struttura trapianto,Città,Organo,Sottotipo,Numero,Anno
0,NO - AOU MAGGIORE DELLA CARITA' - NOVARA,NO,Rene,Rene,37,2024
1,"TO - AOU Città della Salute, PO OIRM",TO,Rene,Rene,4,2024
2,"TO - AOU Città della Salute, PO S.G.Battista",TO,Rene,Rene,194,2024
3,BG - OSPEDALE PAPA GIOVANNI XXIII - BERGAMO,BG,Rene,Rene,44,2024
4,BS - PRES. OSPEDAL. SPEDALI CIVILI BRESCIA,BS,Rene,Rene,60,2024


In [7]:
df_long['Sottotipo'].unique()

array(['Rene', 'Rene doppio', 'Rene - fegato', 'Rene - pancreas',
       'Rene doppio - fegato'], dtype=object)